In [10]:
import boto3
import json
import re
import pandas as pd 
import os 
import io
import pickle

from llm_agent import generate_prompt, call_claude, call_llama
from input import DataInputTest


## Example

In [2]:
   
# === Example ===
viewed_asin_list = ["A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10"]
asin_to_description = {
    "A1": "Noise-cancelling wireless headphones with 30-hour battery life",
    "A2": "Bluetooth speaker with deep bass and waterproof design",
    "A10": "Smart home security camera with motion detection",
    "B1": "Over-ear studio headphones with premium sound",
    "B2": "High-fidelity bookshelf speakers with rich acoustic output"
}
candidate_asins = ("B1", "B2")

prompt = generate_prompt(viewed_asin_list, asin_to_description, *candidate_asins)
response = call_claude(prompt)
# reponse = call_llama(prompt)

print("=== Prompt ===")
print(prompt)
print("\n=== LLM Recommendation ===")
print(response)


=== Prompt ===
You are a smart recommendation assistant.

Here is the customer's recent browsing history:
1. Noise-cancelling wireless headphones with 30-hour battery life
2. Bluetooth speaker with deep bass and waterproof design
3. No description available.
4. No description available.
5. No description available.
6. No description available.
7. No description available.
8. No description available.
9. No description available.
10. Smart home security camera with motion detection

Now, we want to recommend **one** of the following two items:

B1: Over-ear studio headphones with premium sound

B2: High-fidelity bookshelf speakers with rich acoustic output

Based on the customer's interest, which item would you recommend? Respond in the format:

Recommendation: [B1 or B2]


=== LLM Recommendation ===
Recommendation: B1


In [3]:
match = re.search(r"Recommendation:\s*([AB])", response)
if match:
    recommended_option = match.group(1)
    print(f"Recommended option: {recommended_option}")
else:
    print("Could not find recommendation.")

Recommended option: B


## LLM Recommendation

In [9]:
# Loading data
print('Loading data..', flush=True)
s3 = boto3.client('s3')
bucket = 'search-page-template-datasets-devo'
s3_key = 'playground/liangsiq/LLM/LLM_input_data/Digital_Music_dataset.pkl'
buffer = io.BytesIO()

# Download the pickle file into memory
s3.download_fileobj(bucket, s3_key, buffer)
buffer.seek(0)
train_set = pickle.load(buffer)
test_set = pickle.load(buffer)
user_count, item_count, cate_count = pickle.load(buffer)
item_cate_list = pickle.load(buffer)

Loading data..


In [ ]:
# Example of making inference and perform evaluation on 100 samples
record = pd.DataFrame({}, columns=['y', 'recommended'])

for i, batch in DataInputTest(test_set, 1, 10):
    if i > 100: 
        break
    print(i)
    u = batch[0]
    u_cate= batch[12]
    ri_emb= batch[1]
    i= batch[2]
    rj_emb = batch[3] 
    j = batch[4]
    hist_r_emb= batch[5]
    hist_i= batch[6]
    hist_r_new_emb= batch[7]
    hist_i_new= batch[8]
    hist_t= batch[9]
    sl= batch[10]
    sl_new= batch[11]
    
    # Create the mapping
    asin_to_description = dict(zip(hist_i.flatten().tolist(), hist_r_emb[0]))
    asin_to_description[i[0]] = ri_emb 
    asin_to_description[j[0]] = rj_emb

    candidate_asins = (i[0], j[0])
    prompt = generate_prompt(hist_i.flatten().tolist(), asin_to_description, *candidate_asins)
    response = call_claude(prompt)
    # reponse = call_llama(prompt)
    match = re.search(r"Recommendation:\s*(.*)", response)
    if match:
        recommended_option = match.group(1)
        print(f"Recommended option: {recommended_option}")
    else:
        print("Could not find recommendation.")
    record.loc[len(record)] = {'y': i[0], 'recommended': recommended_option}
auc = record[record.y == record.recommended.astype(int)].shape[0]/record.shape[0]